In [1]:
import cv2
import json
import numpy as np
from tqdm import tqdm
import os
import os.path as osp
import multiprocessing
from functools import partial
import glob
import argparse
import IPython
import matplotlib.pyplot as plt
import pandas as pd
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

2023-03-23 17:37:30.357828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 17:37:36.224328: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-23 17:37:46.220963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dat09/venv3.7/lib/python3.7/site-packages/cv2/../../lib64:/home/dat09/TensorRT-8.

In [3]:
with open("../data/AIC23_Track2_NL_Retrieval/data/test-tracks.json") as f:
    tracks_test = json.load(f)
with open("../data/AIC23_Track2_NL_Retrieval/data/train-tracks.json") as f:
    tracks_train = json.load(f)

In [21]:
matcher = Matcher(nlp.vocab)
pattern = [
    [{"POS": "VERB", "op": "+"}, {"LOWER": "straight"}],
    [{"POS": "VERB", "op": "+"}, {"LOWER": "left"}],
    [{"POS": "VERB", "op": "+"}, {"LOWER": "right"}],
    [{"POS": "VERB", "op": "+"}, {"LOWER": "ahead"}],
    [{"LOWER": "stop"}],
    [{"LOWER": "stops"}],   
          ]
matcher.add("motion-chunks", pattern)
def get_motion(text):
    doc = nlp(text)
    matches = matcher(doc)
    output = None
    for match_id, start, end in matches:
        span = doc[start:end]
        output = span.text
        break
    return output

## Visualize direction

In [23]:
save_heatmap_dir = osp.join("../data/AIC23_Track2_NL_Retrieval/data", "data/motion_line")
os.makedirs(save_heatmap_dir, exist_ok=True)

def get_motion_line(data_dir, uuid, boxes_list):
    path = os.path.join(data_dir, f'data/motion_map_iou/{uuid}.jpg')
    # print("path: ", path)
    img = cv2.imread(path)
    w, h, c = img.shape
    line_motion_heatmap = np.zeros([w, h, 3], np.uint8)
    d = 100
    points = [(int(box[0] + box[2]/2), int(box[1] + box[3]/2)) for box in boxes_list]
    for id, boxes in enumerate(points):
        if id != 0:
            point1 = points[id-1]
            point2 = points[id]
            cv2.line(line_motion_heatmap, point1, point2, [0,255,255], 10)
    return line_motion_heatmap

for uuid in tracks_train:
    # print("uuid: ", uuid)
    boxes_list = tracks_train[uuid]["boxes"]
    nl = tracks_train[uuid]["nl"][0]
    motion = get_motion(nl)
    text = "straight"
    if motion is not None:
        if "left" in motion:
            text = "left"
        elif "right" in motion:
            text = "right"
    line_motion_heatmap = get_motion_line("../data/AIC23_Track2_NL_Retrieval/data", uuid, boxes_list)
    cv2.putText(line_motion_heatmap, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imwrite(os.path.join(save_heatmap_dir, f'{uuid}.jpg'), line_motion_heatmap)

# extract points list each track

In [14]:
points_list = []
uuids = []
for uuid in tracks_train:
    uuids.append(uuid)
    boxes_list = tracks_train[uuid]["boxes"]
    points = []
    for box in boxes_list:
        x, y = int(box[0] + box[2]/2), int(box[1] + box[3]/2)
        points.append(x)
        points.append(y)
    str_points = ",".join([str(x) for x in points])
    points_list.append(str_points)
print(len(uuids), len(points_list))
df = pd.DataFrame()
df["uuids"] = uuids
df["points_list"] = points_list
df.to_csv("../data/AIC23_Track2_NL_Retrieval/data/train-tracks.csv",index=False)


points_list = []
uuids = []
for uuid in tracks_test:
    uuids.append(uuid)
    boxes_list = tracks_test[uuid]["boxes"]
    points = []
    for box in boxes_list:
        x, y = int(box[0] + box[2]/2), int(box[1] + box[3]/2)
        points.append(x)
        points.append(y)
    str_points = ",".join([str(x) for x in points])
    points_list.append(str_points)
print(len(uuids), len(points_list))
df = pd.DataFrame()
df["uuids"] = uuids
df["points_list"] = points_list
df.to_csv("../data/AIC23_Track2_NL_Retrieval/data/test-tracks.csv",index=False)

2155 2155
184 184


## get average lenght points list

In [15]:
df = pd.read_csv("../data/AIC23_Track2_NL_Retrieval/data/train-tracks.csv")
points_list = []
for line in df['points_list']:
    points = line.split(',')
    points_list.append(points)
# average length of points
length = 0
for points in points_list:
    length += len(points)
print(length/len(points_list))

,uuids,points_list
0,b06c903c-a25d-45fe-b0d5-294f72e34023,"675,744,667,772,661,800,657,819,654,861,647,88..."
1,3a02a86d-154b-4ee2-bd5e-a0811113a9d8,"1493,439,1433,440,1372,442,1300,445,1287,446,1..."
2,eb184f7c-35c7-4f3e-af52-1db57fb087d4,"1815,476,1792,470,1756,458,1748,452,1716,445,1..."
3,abd32535-8acb-49c5-8da4-9e904d263d8c,"498,482,491,492,484,503,470,517,460,525,448,54..."
4,3819f85b-103a-4f0b-b0f1-49e6b2fedb68,"1667,382,1594,383,1519,383,1447,382,1434,383,1..."


2155
['675', '744', '667', '772', '661', '800', '657', '819', '654', '861', '647', '883', '639', '916']


171.8784222737819
